In [4]:
import requests, json
import youtube_dl

In [5]:
import praw

In [6]:
import pandas as pd
import numpy as np

In [4]:
import datetime

In [5]:
reddit = praw.Reddit(client_id="nT7UQTNYJAEIEQ", client_secret="od1zZJ4vv1Cnv3G7aGRrQccbd-XcPw", user_agent='Tarzan9407')


In [6]:
# Downloading gif file with given url 
def downloadGIF(url, filename):
    
    try:    
        last = url.rsplit(".", 1)
        if last[1] == "gifv":
            url = last[0] + '.' + 'gif'
        if last[1] == "gif":
            with open('redditgif/{}.gif'.format(filename), 'wb') as f:
                f.write(requests.get(url).content)
            return filename + ".gif"
        
        else: 
            ydl_opts = {
            'outtmpl': 'redditgif/{}.mp4'.format(filename)
        }
            with youtube_dl.YoutubeDL(ydl_opts) as ydl:
                ydl.download([url])
            return filename + ".mp4"
    except: 
        return "none" 
    

In [7]:
# json file 
# pandas object of json file containing score, urls hyper link, and title
def scrapeSegmentData(ids:np.array, jsonFile:str, reddit:reddit, redditURL:str):
    # Opening file
    request = requests.get(redditURL)
    datas = request.json()
    with open(jsonFile, "w") as write_file:
        json.dump(datas, write_file, indent=4)
    
    # Iterate over all of the collected submissions 
    for section in datas["data"]:
        
        submission = reddit.submission(section['id'])
        url = section["url"]
        
        if submission.author is not None and submission.is_robot_indexable:
            
            filename = downloadGIF(url, section['id'])
            row = np.array([[submission.title, url, section["full_link"], filename]])
            ids = np.append(ids, row, axis = 0)
        
    return ids
        


In [8]:
# creates a pushshift url to scrpae data before and after a certain timeframe
def createPushshit(before, after):
    return "https://api.pushshift.io/reddit/search/submission/?subreddit=reactiongifs&sort=desc&sort_type=created_utc&after={}&before={}&size=500".format(after, before)


In [9]:
def calcDates(currentTime:datetime.datetime):
    before = currentTime
    after = currentTime - datetime.timedelta(days=7)
    return (before, after)
    


In [10]:
def calcEpochs(dateTup:tuple):
    return(int(dateTup[0].timestamp()), int(dateTup[1].timestamp()))

In [11]:
def collectData(currentDate, finalBeforeDate, jsonFile, array, csvFileName, reddit, columnNames):
    curDate = currentDate
    while curDate > finalBeforeDate:
        dates = calcDates(curDate) # calc before and after date times
        epochs = calcEpochs(dates) # calculating respective epochs
        print(dates) 
        pushshiftURL = createPushshit(epochs[0], epochs[1]) # creating pushshift url 
        try:
            array = scrapeSegmentData(array, jsonFile, reddit, pushshiftURL) # adding data to array
        except:
            pass
        curDate = dates[1] # changing before date to after
    yearData = pd.DataFrame(array, columns = columnNames)
    yearData.to_csv(csvFileName)
    return yearData




In [12]:
currentDate = datetime.datetime(2021,4,18,0,0,0)
finalBeforeDate = datetime.datetime(2015,1,1,0,0,0)
jsonFile = "reddit.json"
array = np.empty(shape = (0,4))
csvFileName = "reddit_data/reddit2021Data.csv"
columns = ["title", 'url', 'full_link', 'filenames']

In [13]:
df = collectData(currentDate, finalBeforeDate, jsonFile, array, csvFileName, reddit, columns)

           
[Imgur] 31BA5dt: Downloading webpage
[download] Destination: redditgif\30o0yb.mp4
[download] 100% of 420.38KiB in 00:00                   
[Imgur] AL7cMTu: Downloading webpage
[download] Destination: redditgif\30nt6q.mp4
[download] 100% of 2.64MiB in 00:00                  
[Imgur] 7cK5iTg: Downloading webpage
[download] Destination: redditgif\30n75f.mp4
[download] 100% of 127.46KiB in 00:00                  
[Gfycat] PleasedCarefreeGoldfinch: Downloading video info
[download] Destination: redditgif\30n3pn.mp4
[download] 100% of 262.23KiB in 00:00                  
[Imgur] hO1OegU: Downloading webpage
[download] Destination: redditgif\30n35v.mp4
[download] 100% of 71.86KiB in 00:00                   
[Imgur] ffqDNDF: Downloading webpage
[download] Destination: redditgif\30n31r.mp4
[download] 100% of 124.02KiB in 00:00                  
[Gfycat] HoarseBonyHydatidtapeworm: Downloading video info
[download] Destination: redditgif\30mun3.mp4
[download] 100% of 71.67KiB in 00:00

In [254]:
df['title'] == "MRW I learned this is Robert Redford in a film"

,Unnamed: 0,title,url,full_link,score,filenames
0,0,MRW I learned this is Robert Redford in a film...,https://i.imgur.com/Jgteohr.gif,https://www.reddit.com/r/reactiongifs/comments...,12557,mt4iov
1,1,MRW I suffer from depression and the interview...,https://i.imgur.com/9TDBvYI.gif,https://www.reddit.com/r/reactiongifs/comments...,60,mt0nqt
2,2,MRW I'm in the waiting room at the sperm bank,https://i.redd.it/l6438iocsst61.gif,https://www.reddit.com/r/reactiongifs/comments...,137,msz066
3,3,"MRW a coworker hopes that ""work gets busy.""",https://i.redd.it/c00pwf5fqst61.gif,https://www.reddit.com/r/reactiongifs/comments...,14,msyt6a
4,4,MRW a police officer says that if journalists ...,https://i.imgur.com/a8c4Jmc.gif,https://www.reddit.com/r/reactiongifs/comments...,112,msy15j
...,...,...,...,...,...,...
713,713,MRW My mom and sister start screaming at each ...,https://i.redd.it/ylbioqbkgq861.gif,https://www.reddit.com/r/reactiongifs/comments...,7856,kocbsl
714,714,MRW reading anything in r/HumanoidEncounters,https://gfycat.com/recentwananemone,https://www.reddit.com/r/reactiongifs/comments...,26,koc6vk
715,715,MRW I go to turn the seat heater in my truck o...,https://i.imgur.com/KHsZHVk.gif,https://www.reddit.com/r/reactiongifs/comments...,27,ko7vn2
716,716,MRW its 2021 and I still see Trump flags and s...,https://i.redd.it/ef6exm4xwn861.gif,https://www.reddit.com/r/reactiongifs/comments...,5252,ko6cje


In [27]:
df

NameError: name 'df' is not defined

In [95]:
reddit.submission("msz066").created_utc


1618693174.0

In [29]:
Submission.permalink


NameError: name 'Submission' is not defined

In [19]:
string = 'dshfjksd.it'

In [16]:
url = "https://imgur.com/a/wicHUWR.gif"

In [17]:
(url.find("imgur") != -1 or url.find("redd.") != -1 or url.find("giphy") != -1 or url.find("gfycat") != -1 or url.find("gif") or url.find("gifv") != -1)

True

In [23]:
df

,title,url,full_link,filenames


In [89]:
request = requests.get("https://api.pushshift.io/reddit/search/submission/?subreddit=reactiongifs&sort=desc&sort_type=created_utc&after=161859000&before=1618590798&size=500")
datas = request.json()
with open(jsonFile, "w") as write_file:
    json.dump(datas, write_file, indent=4)

# Iterate over all of the collected submissions 
for data in datas['data']:
        
    submission = reddit.submission(data['id'])
    url = data["url"]
    print(data)

42c67be39d6dd19242606137eb677dd896bf30e7', 'width': 108}, {'height': 121, 'url': 'https://external-preview.redd.it/6_bQCFDyR6aL2E7yJNMm_a8jiVNGf4XRAGgQ2DQG3ck.gif?width=216&amp;crop=smart&amp;s=a8ae0c2b302dd5d59ea7676833853d4edad01bc0', 'width': 216}, {'height': 180, 'url': 'https://external-preview.redd.it/6_bQCFDyR6aL2E7yJNMm_a8jiVNGf4XRAGgQ2DQG3ck.gif?width=320&amp;crop=smart&amp;s=864e925540a9d7c7ec48a00d17a9a12264b02b23', 'width': 320}, {'height': 360, 'url': 'https://external-preview.redd.it/6_bQCFDyR6aL2E7yJNMm_a8jiVNGf4XRAGgQ2DQG3ck.gif?width=640&amp;crop=smart&amp;s=617654341cc2489314aef0e9a38cfb22b7e3b739', 'width': 640}, {'height': 540, 'url': 'https://external-preview.redd.it/6_bQCFDyR6aL2E7yJNMm_a8jiVNGf4XRAGgQ2DQG3ck.gif?width=960&amp;crop=smart&amp;s=6b72325abf433eccdefefc1b3f3ae5f8c8c2f589', 'width': 960}, {'height': 607, 'url': 'https://external-preview.redd.it/6_bQCFDyR6aL2E7yJNMm_a8jiVNGf4XRAGgQ2DQG3ck.gif?width=1080&amp;crop=smart&amp;s=fa7cf881838ee0ef9fa1d2f715869

In [90]:
datas

rmat=mp4&amp;s=08f8575eabaa4ed2c1738ffe3379ce8f597e3456',
          'width': 960},
         {'height': 607,
          'url': 'https://external-preview.redd.it/6_bQCFDyR6aL2E7yJNMm_a8jiVNGf4XRAGgQ2DQG3ck.gif?width=1080&amp;format=mp4&amp;s=1ec4cd357f9942f873dcb41b60f147bfd34a4e85',
          'width': 1080}],
        'source': {'height': 720,
         'url': 'https://external-preview.redd.it/6_bQCFDyR6aL2E7yJNMm_a8jiVNGf4XRAGgQ2DQG3ck.gif?format=mp4&amp;s=c8e0e29270da7b716853839f1604f299fbc659e7',
         'width': 1280}}}}]},
   'pwls': 6,
   'retrieved_on': 1617972230,
   'score': 1,
   'selftext': '',
   'send_replies': True,
   'spoiler': False,
   'stickied': False,
   'subreddit': 'reactiongifs',
   'subreddit_id': 't5_2t5y3',
   'subreddit_subscribers': 3580986,
   'subreddit_type': 'public',
   'thumbnail': 'https://b.thumbs.redditmedia.com/tpWkMeRFvaxIU4F_0uL4-JVQum3zq132yVDfoMHO1QY.jpg',
   'thumbnail_height': 78,
   'thumbnail_width': 140,
   'title': 'MRW Reddit becomes obses

In [3]:
url = "dfdfdf.fdd.dfdf"

In [12]:
df = pd.read_csv('reddit_data/reddit2021Data.csv')


In [28]:
count = 0
for x in list(df['filenames']):
    y = x.rsplit('.' ,1)
    if len(y) != 2:
        print(x)
        count += 1
    

none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none
none


In [27]:
count

184

In [19]:
df

,Unnamed: 0,title,url,full_link,filenames
0,0,MRW I learned this is Robert Redford in a film...,https://i.imgur.com/Jgteohr.gifv,https://www.reddit.com/r/reactiongifs/comments...,mt4iov.mp4
1,1,MRW I suffer from depression and the interview...,https://i.imgur.com/9TDBvYI.gif,https://www.reddit.com/r/reactiongifs/comments...,mt0nqt.gif
2,2,MRW I'm in the waiting room at the sperm bank,https://i.redd.it/l6438iocsst61.gif,https://www.reddit.com/r/reactiongifs/comments...,msz066.gif
3,3,"MRW a coworker hopes that ""work gets busy.""",https://i.redd.it/c00pwf5fqst61.gif,https://www.reddit.com/r/reactiongifs/comments...,msyt6a.gif
4,4,MRW a police officer says that if journalists ...,https://i.imgur.com/a8c4Jmc.gif,https://www.reddit.com/r/reactiongifs/comments...,msy15j.gif
...,...,...,...,...,...
10283,10283,MRW the neighbor is having a party,http://i.imgur.com/HrlWscd.gif,https://www.reddit.com/r/reactiongifs/comments...,2r8x12.gif
10284,10284,MRW my Ex says that the new Netflix password i...,http://i.imgur.com/et54mtQ.gif,https://www.reddit.com/r/reactiongifs/comments...,2r8uwx.gif
10285,10285,MRW I'm watching two extremely good ping pong ...,http://i.imgur.com/0oOGNuu.gif,https://www.reddit.com/r/reactiongifs/comments...,2r8uhu.gif
10286,10286,MRW I feel myself starting to like something I...,http://i.imgur.com/T5fneZP.jpg,https://www.reddit.com/r/reactiongifs/comments...,2r8svq.mp4


In [30]:
df.filenames = df[df.filenames != 'none']

In [31]:
del df['Unnamed: 0']

In [32]:
df

,title,url,full_link,filenames
0,MRW I learned this is Robert Redford in a film...,https://i.imgur.com/Jgteohr.gifv,https://www.reddit.com/r/reactiongifs/comments...,0
1,MRW I suffer from depression and the interview...,https://i.imgur.com/9TDBvYI.gif,https://www.reddit.com/r/reactiongifs/comments...,1
2,MRW I'm in the waiting room at the sperm bank,https://i.redd.it/l6438iocsst61.gif,https://www.reddit.com/r/reactiongifs/comments...,2
3,"MRW a coworker hopes that ""work gets busy.""",https://i.redd.it/c00pwf5fqst61.gif,https://www.reddit.com/r/reactiongifs/comments...,3
4,MRW a police officer says that if journalists ...,https://i.imgur.com/a8c4Jmc.gif,https://www.reddit.com/r/reactiongifs/comments...,4
...,...,...,...,...
10283,MRW the neighbor is having a party,http://i.imgur.com/HrlWscd.gif,https://www.reddit.com/r/reactiongifs/comments...,10283
10284,MRW my Ex says that the new Netflix password i...,http://i.imgur.com/et54mtQ.gif,https://www.reddit.com/r/reactiongifs/comments...,10284
10285,MRW I'm watching two extremely good ping pong ...,http://i.imgur.com/0oOGNuu.gif,https://www.reddit.com/r/reactiongifs/comments...,10285
10286,MRW I feel myself starting to like something I...,http://i.imgur.com/T5fneZP.jpg,https://www.reddit.com/r/reactiongifs/comments...,10286
